# Семинар 4.
## Линейная алгебра и нейросети

## Нейросеть как суперпозиция функций

- Линейных и нелинейных
- Можно представить в виде графа вычислений
- С помощью этого графа можно [автоматически вычислять градиенты](https://en.wikipedia.org/wiki/Automatic_differentiation)

## Что такое градиент и как его вычислить?

- Если функция $f$ зависит от вектора или матрицы и вычисляет скаляр, то её градиент по аргументу - это вектор или матрица, элементы которых – производные выхода по соответствующему элементу аргумента
- Пример: $f(x) = x^{\top}Ax + b^{\top}x$. Чему равен градиент?
- Поэлементный пример: $f(x) = x^2$ поэлементно. Чему равен градиент (матрица Якоби)?

In [5]:
import torch

n = 5
A = torch.randn((n, n), requires_grad=True)
b = torch.randn((n,))
x = torch.randn((n,), requires_grad=True)

f = 0.5 * x @ A @ x - b @ x
g = f * f
g.backward()
print(f)
print(f.item())

tensor(-3.7053, grad_fn=<SubBackward0>)
-3.70530104637146


In [6]:
manual_grad_x = 0.5 * (A + A.t()) @ x - b

print(manual_grad_x.data)
print(x.grad.data)

tensor([ 0.0102, -0.7805,  1.1065, -0.3718, -4.2938])
tensor([-0.0755,  5.7839, -8.1999,  2.7555, 31.8199])


In [7]:
manual_grad_A = 0.5 * torch.ger(x, x)

print(manual_grad_A.data)
print(A.grad.data)
print(torch.norm(manual_grad_A.data - A.grad.data).item())

tensor([[ 9.6064e-03, -1.7660e-03,  1.4851e-01, -5.8479e-02, -5.7553e-02],
        [-1.7660e-03,  3.2464e-04, -2.7301e-02,  1.0750e-02,  1.0580e-02],
        [ 1.4851e-01, -2.7301e-02,  2.2959e+00, -9.0405e-01, -8.8974e-01],
        [-5.8479e-02,  1.0750e-02, -9.0405e-01,  3.5598e-01,  3.5035e-01],
        [-5.7553e-02,  1.0580e-02, -8.8974e-01,  3.5035e-01,  3.4481e-01]])
tensor([[-7.1189e-02,  1.3087e-02, -1.1006e+00,  4.3336e-01,  4.2650e-01],
        [ 1.3087e-02, -2.4058e-03,  2.0232e-01, -7.9665e-02, -7.8405e-02],
        [-1.1006e+00,  2.0232e-01, -1.7014e+01,  6.6995e+00,  6.5935e+00],
        [ 4.3336e-01, -7.9665e-02,  6.6995e+00, -2.6381e+00, -2.5963e+00],
        [ 4.2650e-01, -7.8405e-02,  6.5935e+00, -2.5963e+00, -2.5552e+00]])
25.28742790222168


## Причём тут нейросети?

- Обучение с учителем (но не только!)
- Есть данные, есть ответы
- Нейросеть $\approx$ сложная композиция простых функций, которая по данным восстанавливает ответы
- Для обучения необходима целевая функция a.k.a. функция потерь или loss, который минимизируется каким-нибудь стохастическим методом первого порядка
- Поэтому нужны градиенты!

<img src="./pytorch_logo.png">

- Удобный фреймворк для построения и обучения нейросетей
- Реализует динамический граф вычислений
- Вы просто пишете функцию, которая вычисляет нужное вам выражение, вычисляете (вызываете) её в некоторой точке (с некоторым аргументом) и можете получить градиенты в этой точке бесплатно!
- Специальные инструменты для построения нейросетей: модуль с большинством популярных слоёв, для которых градиенты уже реализованы
- Пока плохо поддерживает операции с разреженными матрицами

## GPU vs. CPU

- CPU – мощный вычислитель, но мало памяти доступно здесь и сейчас (см. лекцию про иерархию памяти)
- GPU – ОЧЕНЬ много ядер для выполнения простых операций
- CPU распределяет задачи по ядрам, GPU - по кластерам ядер

Только использование GPU (или [TPU](https://habr.com/ru/post/422317/)) позволит за разумное время обучить сложные модели для задач компьютерного зрения, NLP, RL и других областей.

### Какие операции нужны для работы нейросетей?

- Умножение матрицы на вектор и векторное сложение - полносвязный слой
- Поэлементые операции - нелинейности
- Локальные операции типа [MaxPooling](https://deepai.org/machine-learning-glossary-and-terms/max-pooling)
- Получение из вектора или матрицы скаляра

### Пример реализации операции матричного умножения

- Пример исходного кода, реализующего матричное умножение на CUDA, доступен [тут](https://www.quantstart.com/articles/Matrix-Matrix-Multiplication-on-the-GPU-with-Nvidia-CUDA)
- Сравнение с помощью PyTorch доступно по ссылке ниже [![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1tHGFnvgoKnrq2C955wwdfloARnqTNsBr?usp=sharing)

## Резюме

- В основе нейросетей как и других вычислительных технологий лежит линейная алгебра
- Современные фреймворки (PyTorch, etc) позволяют вычислять градиенты автоматически
- GPU быстрее CPU для выполнения большого числа простых операций
- TPU ещё быстрее, но менее доступны
- [Colab](https://colab.research.google.com/) позволяет экспериментировать с этими технологиями удалённо